In [ ]:
from src.data import make_dataset
from transformers import pipeline
from sklearn.metrics import classification_report
import torch
import pandas as pd
import gc

In [ ]:
if torch.cuda.is_available():
    device = 0
else:
    device = -1

In [ ]:
# input_file = "../../data/raw/reviews.csv"
# train_output_file = "../../data/processed/train_final_processed_reviews.csv"
# test_output_file = "../../data/processed/test_final_processed_reviews.csv"
# X_train, X_test, y_train, y_test = make_dataset.main(input_file, train_output_file, test_output_file)


In [ ]:
train = pd.read_csv("../../data/processed/train_final_processed_reviews.csv", index_col='Unnamed: 0')
test = pd.read_csv("../../data/processed/test_final_processed_reviews.csv", index_col='Unnamed: 0')
X_train = train.drop('sentiment', axis=1)
X_test = test.drop('sentiment', axis=1)
y_train = train.sentiment
y_test = test.sentiment

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
data = X_test.text.to_list()
data = [x[:512] if len(x)>512 else x for x in data]

In [ ]:
%%time
sentiment_pipeline = pipeline(model = "distilbert-base-uncased-finetuned-sst-2-english", device=device)
results0 = sentiment_pipeline(data)
results0[0:5]

In [ ]:
%%time
specific_model = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)
results1 = specific_model(data)
results1[0:5]

In [ ]:
%%time
specific_model = pipeline(model="Seethal/sentiment_analysis_generic_dataset", device=device)
results2 = specific_model(data)
results2[0:5]

In [ ]:
%%time
specific_model = pipeline('sentiment-analysis', model="siebert/sentiment-roberta-large-english", device=device)
results3 = specific_model(data)
results3[0:5]

# Test Results

In [ ]:
y_true = [1 if label=='positive' else 0 for label in y_test]

## Results for distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
labels = [result['label'] for result in results0]
y_pred = [1 if label=='POSITIVE' else 0 for label in labels]
print(classification_report(y_true, y_pred))

## Results for nlptown/bert-base-multilingual-uncased-sentiment"

In [ ]:
labels = [result['label'] for result in results1]
y_pred = [1 if label in ['5 stars', '4 stars', '3 stars'] else 0 for label in labels]
print(classification_report(y_true, y_pred))

## Results for Seethal/sentiment_analysis_generic_dataset

In [ ]:
labels = [result['label'] for result in results2]
y_pred = [1 if label=='LABEL_2' else 0 for label in labels]
# y_pred = [0 if label=='LABEL_0' else 1 for label in labels]
print(classification_report(y_true, y_pred))

## Results for siebert/sentiment-roberta-large-english

In [ ]:
labels = [result['label'] for result in results3]
y_pred = [1 if label=='POSITIVE' else 0 for label in labels]
print(classification_report(y_true, y_pred))

In [ ]:
results3

In [ ]:
sentiment_pipeline=None
specific_model=None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from src.models.sentiment_analysis.pre_trained.seibert import Seibert

model = Seibert()
pred = model.predict(X_test).predicted_sentiment
print(classification_report(y_true, pred))

In [ ]:
# from src.models.sentiment_analysis.pre_trained.bert_fine_tuned import BertFineTuned
# model = BertFineTuned('bert_state_dict.pt')
# pred = model.predict(X_test)
# print(classification_report(y_true, pred))